In [1]:
!python --version

Python 3.11.11


In [2]:
# Install dependencies
!pip install -q --upgrade numerapi pandas pyarrow matplotlib lightgbm scikit-learn scipy cloudpickle==3.1.1


[notice] A new release of pip is available: 25.0.1 -> 25.2
[notice] To update, run: pip install --upgrade pip


In [8]:
from numerapi import NumerAPI
import pandas as pd
import lightgbm as lgb

napi = NumerAPI()

# use one of the latest data versions
DATA_VERSION = "crypto/v2.0_beta"

# Download and read training data
napi.download_dataset(f"{DATA_VERSION}/train.parquet")
data = pd.read_parquet(f"{DATA_VERSION}/train.parquet")

# split data into train / val at a ~70/30 ratio
split = int(len(data) * 0.7)
train = data[:split].copy()
val = data[split:].copy()

2025-10-07 16:57:38,443 INFO numerapi.utils: target file already exists
2025-10-07 16:57:38,444 INFO numerapi.utils: starting download
crypto/v2.0_beta/train.parquet: 7.15MB [00:00, 22.5MB/s]                   


In [ ]:

train

,date,feature_bollinger_20d,feature_bollinger_60d,feature_close_avg_20d,feature_close_avg_60d,feature_close_ewa_20d,feature_close_ewa_60d,feature_market_cap_avg_20d,feature_market_cap_avg_60d,feature_market_cap_ewa_20d,feature_market_cap_ewa_60d,feature_momentum_20d,feature_momentum_60d,feature_relative_strength_index_20d,feature_relative_strength_index_60d,feature_sharpe_ratio_20d,feature_sharpe_ratio_60d,feature_volatility_20d,feature_volatility_60d,feature_volume_avg_20d,feature_volume_avg_60d,feature_volume_ewa_20d,feature_volume_ewa_60d,target_binned_return_20,target_binned_return_60
symbol,,,,,,,,,,,,,,,,,,,,,,,,,
BTC,2020-01-01,0.50,0.50,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,0.00,0.00,0.75,0.50,0.50,0.50,0.00,0.00,1.00,1.00,1.00,1.00,0.50,0.50
LTC,2020-01-01,0.75,0.50,1.00,1.00,1.00,1.00,1.00,1.00,1.00,1.00,0.00,0.00,0.50,0.50,0.50,0.25,0.25,0.25,1.00,1.00,1.00,1.00,0.75,0.50
XRP,2020-01-01,0.50,0.50,0.50,0.50,0.50,0.50,1.00,1.00,1.00,1.00,0.25,0.25,0.50,0.25,0.50,0.25,0.25,0.00,1.00,1.00,1.00,1.00,0.50,0.50
DOGE,2020-01-01,0.25,0.25,0.25,0.25,0.25,0.25,0.75,0.75,0.75,0.75,0.50,0.50,0.50,0.50,0.50,0.50,0.00,0.00,0.75,0.75,0.75,0.75,0.50,0.50
VTC,2020-01-01,0.25,0.25,0.50,0.50,0.50,0.50,0.50,0.50,0.50,0.50,0.25,0.50,0.25,0.50,0.25,0.50,0.50,0.75,0.25,0.25,0.25,0.25,1.00,0.75
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
SAND,2024-01-01,0.50,0.75,0.50,0.50,0.50,0.50,0.75,0.75,0.75,0.75,0.50,0.50,0.75,0.75,0.75,0.75,0.50,0.25,0.75,0.75,0.75,0.75,0.50,0.50
NEAR,2024-01-01,0.50,0.75,0.50,0.50,0.50,0.50,0.75,0.75,0.75,0.75,0.75,0.75,0.75,0.75,0.75,0.75,0.75,0.50,0.75,0.75,0.75,0.75,0.25,0.75
CRV,2024-01-01,0.25,0.25,0.50,0.50,0.50,0.50,0.50,0.50,0.50,0.50,0.50,0.50,0.50,0.25,0.50,0.25,0.50,0.50,0.75,0.50,0.75,0.50,0.25,0.50


In [10]:
# Train model
model = lgb.LGBMRegressor(
    n_estimators=2000,
    learning_rate=0.01,
    max_depth=5,
    num_leaves=2**5-1,
    colsample_bytree=0.1
)
# We've found the following "deep" parameters perform much better,
#   but they require much more CPU and RAM
# model = lgb.LGBMRegressor(
#     n_estimators=30_000,
#     learning_rate=0.001,
#     max_depth=10,
#     num_leaves=2**10,
#     colsample_bytree=0.1,
#     min_data_in_leaf=10000,
# )
model.fit(
    train.filter(like="feature_"),
    train["target_binned_return_20"]
)

[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000906 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 112
[LightGBM] [Info] Number of data points in the train set: 306947, number of used features: 22
[LightGBM] [Info] Start training from score 0.499964
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain:

,boosting_type,'gbdt'
,num_leaves,31
,max_depth,5
,learning_rate,0.01
,n_estimators,2000
,subsample_for_bin,200000
,objective,None
,class_weight,None
,min_split_gain,0.0
,min_child_weight,0.001
,min_child_samples,20


In [ ]:
from numerai_tools.scoring import numerai_corr

val_predictions = model.predict(val.filter(like="feature_"))
validation_corr = numerai_corr(val_predictions, val["target_binned_return_20"])

In [ ]:
# download and read live data
napi.download_dataset(f"{DATA_VERSION}/live.parquet")
live_data = pd.read_parquet(f"{DATA_VERSION}/live.parquet")

# generate live predictions
live_data["prediction"] = model.predict(live_data.filter(like="feature_"))
live_data[["id", "prediction"]].to_parquet("predictions.parquet", index=False)